# Create the C,A -> R dataset to be used for training classifier

In [1]:
labels = [
    "0",
    "B-sentence",
    "I-sentence",
]
BGN = "[BGN]"
END = "[END]"

In [2]:
# necessary library imports
import pandas as pd
import numpy as np

In [12]:
# data imports, to be combined into the final datastructure
df_train_cleaned = pd.read_pickle("../data_frames/df_train_cleaned.pkl")
df_relevant_sentences = pd.read_pickle("../data_frames/df_relevant_sentences_lemma_stop.pkl")

df_relevant_sentences.head()

,context_stop_lemmas,sent_stop_lemmas,sent_original,q_stop_lemmas,q_original,overlap_class,num_overlap_sentences,missing_words_sent,missing_words_context,matching_sentence_ids,matching_sentences,matching_original_sentences,data,ranked_matching_sentences,ranked_matching_sentence_ids,ranked_matching_original_sentences
0,"[[visa, affär, konferensbesök], [om, skola, be...","[visum, tillstånd, resa, in, land, kort, tid]","[visum, är, ett, tillstånd, för, att, resa, in...",[visum],"[vad, är, ett, visum]",1,5,[],[],"[2, 1, 3, 6, 7]","[visum tillstånd resa in land kort tid, om sko...",[visum är ett tillstånd för att resa in och va...,"{2: {'original_sent_words': ['visum', 'är', 'e...","(visum tillstånd resa in land kort tid, pass g...","(2, 7, 1, 6, 3)",(visum är ett tillstånd för att resa in och va...
1,"[[egen, företag], [efter, beslut, skicka, amba...","[skola, hämta, beslut, ta, pass]","[när, du, ska, hämta, ditt, beslut, ska, du, t...","[skola, ta, hämta, beslut]","[vad, ska, du, ta, med, när, du, hämtar, ditt,...",1,6,[],[],"[2, 1, 6, 9, 11, 12]","[skola hämta beslut ta pass, efter beslut skic...",[när du ska hämta ditt beslut ska du ta med di...,"{2: {'original_sent_words': ['när', 'du', 'ska...","(skola hämta beslut ta pass, uppehållstillstån...","(2, 6, 9, 11, 1, 12)",(när du ska hämta ditt beslut ska du ta med di...
2,"[[utbytesdoktorand, intervju], [du, komma, få,...","[länge, gälla, tillstånde, uppehållstillstånd,...","[så, länge, gäller, tillståndet, ett, uppehåll...","[länge, gälla, uppehållstillstånd, besök]","[hur, länge, gäller, ett, uppehållstillstånd, ...",1,4,[],[],"[18, 4, 5, 19]",[länge gälla tillstånde uppehållstillstånd bes...,[så länge gäller tillståndet ett uppehållstill...,"{18: {'original_sent_words': ['så', 'länge', '...",(länge gälla tillstånde uppehållstillstånd bes...,"(18, 19, 4, 5)",(så länge gäller tillståndet ett uppehållstill...
3,"[[egen, företag], [när, ta, emot, ansökan], [o...","[kort, bevis, tillstånd, sverige, innehålla, b...","[kortet, är, ett, bevis, på, att, du, har, til...",[uppehållstillståndskort],"[vad, är, ett, uppehållstillståndskort]",2,3,[uppehållstillståndskort],[],"[8, 7, 10]",[kort bevis tillstånd sverige innehålla bland ...,[kortet är ett bevis på att du har tillstånd a...,"{8: {'original_sent_words': ['kortet', 'är', '...",(om få tillstånd mycket tre månad uppehållstil...,"(7, 10, 8)",(om du får tillstånd för mer än tre månader få...
4,"[[flytta, tillbaka, Sverige], [så, ansöka, du,...","[kort, bevis, tillstånd, sverige, innehålla, b...","[kortet, är, ett, bevis, på, att, du, har, til...",[uppehållstillståndskort],"[vad, är, ett, uppehållstillståndskort]",2,5,[uppehållstillståndskort],[],"[8, 7, 10, 12, 20]",[kort bevis tillstånd sverige innehålla bland ...,[kortet är ett bevis på att du har tillstånd a...,"{8: {'original_sent_words': ['kortet', 'är', '...",(om få tillstånd mycket tre månad uppehållstil...,"(7, 20, 12, 10, 8)",(om du får tillstånd för mer än tre månader få...


In [17]:
def find_answer_start(answer, sent):
    print('answer: ', answer)
    print('sent: ', sent)
    for idx, word in enumerate(sent):
        if answer[0] in word:
            is_match = True
            for i in range(len(answer)):
                if idx+i < len(sent) and answer[i] not in sent[idx+i]:
                    is_match = False
            if is_match:
                return idx

    return None

In [18]:
def get_tokens_and_labels(sentences, ranked_sentence_ids, answer):
    context_text = []
    all_labels = []
    sent_match_id = ranked_sentence_ids[0]

    # get the labels and tokens for current sentence
    for idx, sent in enumerate(sentences):
        if idx in ranked_sentence_ids:
            c_sent = sent.copy()
            c_sent.insert(0, BGN)
            c_sent.append(END)
            context_text += c_sent
            labels = np.zeros(len(c_sent))
            # TODO: mark the correct answer in the context text
            if idx == sent_match_id:
                idx_s = find_answer_start(answer, c_sent)
                if idx_s:
                    labels[idx_s] = 1
                    for i in range(len(answer)-1):
                        labels[idx_s + i + 1] = 2
                else:
                    print('did not find ans in sent!')

        else:
            context_text += sent # concatenate all sentences to a list of consecutive tokens
            labels = np.zeros(len(sent))

        all_labels.append(labels)
    l = np.concatenate(all_labels).ravel()
    return context_text, l


In [19]:
# create the dataset with the corresponding labels
def label_data(df, df_relevant_sentences):
    relevant_sentence_data = []

    for index, row in df.iterrows():
        sentences = row['context_raw']
        answer = row['correct_answer_raw']
        sent_with_ans_id = row['answer_location']
        relevant_sentence_ids = df_relevant_sentences.iloc[index]['ranked_matching_sentence_ids']
        sent_ids = [sent_with_ans_id]
        
        count = 0
        # add (max 3 including sentence with answer) highest ranked sententces
        for sent_id in relevant_sentence_ids:
            if count < 2 and sent_id != sent_with_ans_id:
                sent_ids.append(sent_id)
                count += 1

        context_text, labels = get_tokens_and_labels(sentences, sent_ids, answer)
        labels = [ int(x) for x in labels ]
        data_point = { 'id': index, 'labels': labels, 'tokens': context_text, 'answer': answer }
        relevant_sentence_data.append(data_point)
    
    

    print('num data points: ', len(relevant_sentence_data))
    return relevant_sentence_data


In [21]:
labeled_data = label_data(df_train_cleaned, df_relevant_sentences)
labeled_df = pd.DataFrame(labeled_data)
labeled_df.to_pickle("./data/labeled_CRA_training_data.pkl")
# print(labeled_df[0]['tokens'])
labeled_df.head()


answer:  ['ett', 'tillstånd', 'för', 'att', 'resa', 'in', 'och', 'vara', 'i', 'ett', 'land']
sent:  ['[BGN]', 'Visum', 'är', 'ett', 'tillstånd', 'för', 'att', 'resa', 'in', 'och', 'vara', 'i', 'ett', 'land', 'under', 'en', 'kortare', 'tid', '.', '[END]']
answer:  ['ditt', 'pass']
sent:  ['[BGN]', 'När', 'du', 'ska', 'hämta', 'ditt', 'beslut', 'ska', 'du', 'ta', 'med', 'ditt', 'pass', '.', '[END]']
answer:  ['ett', 'år']
sent:  ['[BGN]', 'Så', 'länge', 'gäller', 'tillståndet', 'Ett', 'uppehållstillstånd', 'för', 'besök', 'gäller', 'som', 'längst', 'upp', 'till', 'ett', 'år', ',', 'men', 'oftast', 'högst', 'sex', 'månader', '.', '[END]']
answer:  ['ett', 'bevis']
sent:  ['[BGN]', 'Kortet', 'är', 'ett', 'bevis', 'på', 'att', 'du', 'har', 'tillstånd', 'att', 'vara', 'i', 'Sverige', 'och', 'innehåller', 'bland', 'annat', 'foto', 'på', 'dig', 'och', 'dina', 'fingeravtryck', '.', '[END]']
answer:  ['ett', 'bevis']
sent:  ['[BGN]', 'Kortet', 'är', 'ett', 'bevis', 'på', 'att', 'du', 'har', 'til

,id,labels,tokens,answer
0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Visum, för, affärs-, och, konferensbesök, [BG...","[ett, tillstånd, för, att, resa, in, och, vara..."
1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Eget, företag, Efter, beslut, Beslutet, skick...","[ditt, pass]"
2,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Utbytesdoktorander, Intervju, Du, kommer, att...","[ett, år]"
3,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Eget, företag, När, vi, har, tagit, emot, din...","[ett, bevis]"
4,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Flytta, tillbaka, till, Sverige, Så, ansöker,...","[ett, bevis]"


In [54]:
# TODO: prepare the test data
# labeled_test_data = label_data(df_test)
# labeled_test_df = pd.DataFrame(labeled_test_data)
# labeled_test_df.to_pickle("./data/labeled_sentence_extraction_test_data.pkl")